<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/Sentence_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install sentence transformer
!pip install -U sentence-transformers

In [2]:
# import sentence transformer class
from sentence_transformers import SentenceTransformer,util
model = SentenceTransformer('all-MiniLM-L6-v2')

## Embedding example

In [ ]:
# write down example sentences
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.']

# Using model to encode into sentence embeddings
embeddings = model.encode(sentences)


for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

## Cosine Sentence Similarity

In [6]:
# Single list of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

#Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)
cosine_scores

tensor([[ 1.0000,  0.0363,  0.0081, -0.0247,  0.6788,  0.1310, -0.0029,  0.0254],
        [ 0.0363,  1.0000, -0.0368,  0.0093,  0.2105, -0.0327, -0.0136,  0.0116],
        [ 0.0081, -0.0368,  1.0000,  0.2440,  0.0230,  0.0359,  0.2560,  0.5096],
        [-0.0247,  0.0093,  0.2440,  1.0000,  0.0275, -0.0502,  0.8939,  0.1969],
        [ 0.6788,  0.2105,  0.0230,  0.0275,  1.0000,  0.0629,  0.0591,  0.0900],
        [ 0.1310, -0.0327,  0.0359, -0.0502,  0.0629,  1.0000, -0.0509,  0.0417],
        [-0.0029, -0.0136,  0.2560,  0.8939,  0.0591, -0.0509,  1.0000,  0.1692],
        [ 0.0254,  0.0116,  0.5096,  0.1969,  0.0900,  0.0417,  0.1692,  1.0000]])

In [24]:
# List the cosine similarity between each pairs
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
pairs

[{'index': [3, 6], 'score': tensor(0.8939)},
 {'index': [0, 4], 'score': tensor(0.6788)},
 {'index': [2, 7], 'score': tensor(0.5096)},
 {'index': [2, 6], 'score': tensor(0.2560)},
 {'index': [2, 3], 'score': tensor(0.2440)},
 {'index': [1, 4], 'score': tensor(0.2105)},
 {'index': [3, 7], 'score': tensor(0.1969)},
 {'index': [6, 7], 'score': tensor(0.1692)},
 {'index': [0, 5], 'score': tensor(0.1310)},
 {'index': [4, 7], 'score': tensor(0.0900)},
 {'index': [4, 5], 'score': tensor(0.0629)},
 {'index': [4, 6], 'score': tensor(0.0591)},
 {'index': [5, 7], 'score': tensor(0.0417)},
 {'index': [0, 1], 'score': tensor(0.0363)},
 {'index': [2, 5], 'score': tensor(0.0359)},
 {'index': [3, 4], 'score': tensor(0.0275)},
 {'index': [0, 7], 'score': tensor(0.0254)},
 {'index': [2, 4], 'score': tensor(0.0230)},
 {'index': [1, 7], 'score': tensor(0.0116)},
 {'index': [1, 3], 'score': tensor(0.0093)},
 {'index': [0, 2], 'score': tensor(0.0081)},
 {'index': [0, 6], 'score': tensor(-0.0029)},
 {'index'

In [25]:
for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))

The new movie is awesome 		 The new movie is so great 		 Score: 0.8939
The cat sits outside 		 The cat plays in the garden 		 Score: 0.6788
I love pasta 		 Do you like pizza? 		 Score: 0.5096
I love pasta 		 The new movie is so great 		 Score: 0.2560
I love pasta 		 The new movie is awesome 		 Score: 0.2440
A man is playing guitar 		 The cat plays in the garden 		 Score: 0.2105
The new movie is awesome 		 Do you like pizza? 		 Score: 0.1969
The new movie is so great 		 Do you like pizza? 		 Score: 0.1692
The cat sits outside 		 A woman watches TV 		 Score: 0.1310
The cat plays in the garden 		 Do you like pizza? 		 Score: 0.0900


## Semantic Search

In [26]:
modl = SentenceTransformer('clips/mfaq')


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [27]:
# Sample question and answers
question = "How many models can I host on HuggingFace?"
answer_1 = "All plans come with unlimited private models and datasets."
answer_2 = "AutoNLP is an automatic way to train and deploy state-of-the-art NLP models, seamlessly integrated with the Hugging Face ecosystem."
answer_3 = "Based on how much training data and model variants are created, we send you a compute cost and payment link - as low as $10 per job."

query_embedding = model.encode(question)
corpus_embeddings = model.encode([answer_1, answer_2, answer_3])

print(util.semantic_search(query_embedding, corpus_embeddings))

[[{'corpus_id': 0, 'score': 0.35359811782836914}, {'corpus_id': 1, 'score': 0.3143519163131714}, {'corpus_id': 2, 'score': 0.24975530803203583}]]


## Clustering using sentence transformers

In [28]:
from sklearn.cluster import KMeans
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'Horse is eating grass.',
          'A man is eating pasta.',
          'A Woman is eating Biryani.',
          'The girl is carrying a baby.',
          'The baby is carried by the woman',
          'A man is riding a horse.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.',
          'A cheetah is running behind its prey.',
          'A cheetah chases prey on across a field.',
          'The cheetah is chasing a man who is riding the horse.',
          'man and women with their baby are watching cheetah in zoo'
          ]
corpus_embeddings = embedder.encode(corpus)

In [29]:
corpus_embeddings

array([[ 0.03324159,  0.00440611, -0.00627698, ...,  0.06917298,
        -0.02458487, -0.03756437],
       [ 0.05251842,  0.05524131, -0.0111882 , ..., -0.01619394,
        -0.06022897, -0.04115053],
       [-0.00858896,  0.01801807,  0.05754678, ...,  0.06797205,
         0.0076367 , -0.03118757],
       ...,
       [ 0.05438586,  0.05397905, -0.0036798 , ...,  0.03249276,
         0.02187709,  0.06213458],
       [-0.00162545,  0.04804113,  0.0103358 , ..., -0.00226648,
         0.09303317,  0.03106402],
       [ 0.03091299,  0.00874069, -0.06028781, ...,  0.04663473,
         0.03548018, -0.00794349]], dtype=float32)

In [30]:
len(corpus_embeddings)

15

In [ ]:
# sample from corpus
corpus_embeddings[1]

In [32]:
# let's fit K-means model to clusters
clustering_model = KMeans(n_clusters=3)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)

[2 2 1 2 2 0 0 1 1 0 0 1 1 1 0]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [33]:
# Create cluster of the sentence
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
clustered_sentences

{2: ['A man is eating food.',
  'A man is eating a piece of bread.',
  'A man is eating pasta.',
  'A Woman is eating Biryani.'],
 1: ['Horse is eating grass.',
  'A man is riding a horse.',
  'A man is riding a white horse on an enclosed ground.',
  'A cheetah is running behind its prey.',
  'A cheetah chases prey on across a field.',
  'The cheetah is chasing a man who is riding the horse.'],
 0: ['The girl is carrying a baby.',
  'The baby is carried by the woman',
  'A monkey is playing drums.',
  'Someone in a gorilla costume is playing a set of drums.',
  'man and women with their baby are watching cheetah in zoo']}